### Downloading Images

In [ ]:
!conda env create -f airnet.yaml

In [9]:
!mkdir -p dataset/images
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/test.zip

--2024-04-18 06:01:26--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11298421598 (11G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  10.52G  22.9MB/s    in 7m 54s  

2024-04-18 06:09:20 (22.7 MB/s) - ‘train.zip’ saved [11298421598/11298421598]

--2024-04-18 06:09:20--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip
Reusing existing connection to vizwiz.cs.colorado.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 3488913457 (3.2G) [application/zip]
Saving to: ‘val.zip’

val.zip             100%[===================>]   3.25G  22.5MB/s    in 2m 26s  

2024-04-18 06:11:46 (22.7 MB/s) - ‘val.zip’ saved [3488913457/3488913457]

--2024-04-18 06:11:46--  https://vizwiz.cs.colorado.edu/VizWi

In [10]:
!unzip -q -o train.zip -d dataset/images
!unzip -q -o val.zip -d dataset/images
!unzip -q -o test.zip -d dataset/images

In [11]:
!rm train.zip val.zip test.zip

### Airnet

In [14]:
!git clone https://github.com/XLearning-SCU/2022-CVPR-AirNet

Cloning into '2022-CVPR-AirNet'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 124 (delta 4), reused 3 (delta 3), pack-reused 112
Receiving objects: 100% (124/124), 36.54 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [12]:
# Hopefully you won't need any extra imports but if you do it shouldn't be too hard I hope
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
    ])

In [ ]:
####################################
# I HAVE NO IDEA IF THIS WORKS     #
# THIS IS SOME RANDOM CHATGPT CODE #
####################################
!mkdir airnet
!mkdir transformed_images

# Praise ChatGPT
input_dir = 'dataset/images/'
output_dir = ''

shutil.rmtree(output_dir, ignore_errors=True)
shutil.copytree(input_dir, output_dir)

for root, dirs, files in os.walk(output_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Open the image
            img = Image.open(os.path.join(root, file))

            # Apply the transformation pipeline
            transformed_img = test_transform(img)

            # Save the transformed image, preserving the folder structure
            output_subdir = root.replace(output_dir, input_dir)
            os.makedirs(output_subdir, exist_ok=True)
            transformed_img.save(os.path.join(output_subdir, file))

In [23]:
!python 2022-CVPR-AirNet/demo.py --test_path='./dataset/images' --output_path='./airnet' --mode 3

Traceback (most recent call last):
  File "2022-CVPR-AirNet/demo.py", line 4, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'
